In [2]:
#FIN 553 Group Project 1 - Part 1
#Takumi Li (feiyang3), Pengjie Ren (pengjie2), Wenqing Ni(wn5)

import tensorflow as tf
import numpy as np
Spot = 36   # stock price
σ = 0.2     # stock volatility
K = 40      # strike price
r = 0.06    # risk free rate
n = 100000  # Number of simualted paths
m = 50      # number of exercise dates
T = 1       # maturity
order = 6   # Polynmial order
Δt = T / m  # interval between two exercise dates

In [3]:
# Construct polynomial features of order up to k using the
# recursive formulation
def chebyshev_basis(x, k):  #x = [0,1,2]
    B = [tf.ones(len(x)), x]   #B = [[1,1,1],[0,1,2]]
    for n in range(2, k):
        Bn = 2 * x * B[n - 1] - B[n - 2] # 2*[0,1,2]*[0,1,2] - [1,1,1]
        B.append(Bn)    # [-1, 1, 7]   

    return tf.stack(B,axis = 1)


In [4]:
# scales x to be in the interval(-1, 1)
def scale(x):
    xmin = tf.reduce_min(x)
    xmax = tf.reduce_max(x)
    a = 2 / (xmax - xmin)
    b = 1 - a * xmax
    return a * x + b

In [5]:
# simulates the stock price evolution
def advance(S, r, σ, Δt, n):
    dB = tf.sqrt(Δt) * tf.random.normal([len(S)])
    out = S + r * S * Δt + σ * S * dB
    return out

In [7]:
# LSMC algorithm

tf.random.set_seed(0)
S0 = Spot * tf.ones(n)
S = {0: S0}

for t in range(m):
    S[t + 1] = advance(S[t], r, σ, Δt, n)

discount = tf.exp(-r * Δt)
CFL = {t: tf.maximum(0., K - S[t]) for t in S.keys()}
value_tp1 = CFL[m] * discount # array 10w array
CV = {m: tf.zeros_like(S0)}

for t in range(m - 1, 0, -1):
    X = chebyshev_basis(scale(S[t]), order) # 6 * 10W MATRIX
    Y = tf.constant(value_tp1, shape=(100000, 1)) # 10w * 1



    # regression to estimate the continuation value
    Θ = tf.linalg.solve(tf.matmul(tf.transpose(X),X),tf.matmul(tf.transpose(X),Y))    # 6* 10w  *  10 w * 6    6 * 10w * 10w * 1    theta 6*1
    CV[t] = tf.matmul(X,Θ)
    CFL[t] = tf.reshape(CFL[t],[100000,1])
    
    CV[t] = tf.convert_to_tensor(CV[t],dtype=tf.float32)
    CFL[t] = tf.convert_to_tensor(CFL[t],dtype=tf.float32)

    
    exercise = tf.math.greater(CFL[t] , CV[t])
    value_tp1 = tf.reshape(value_tp1, [100000,1])
    
    value_t = tf.where(exercise, CFL[t], value_tp1)
    
    value_tp1 = discount * value_t

POF = {t: tf.where(CV[t] < CFL[t], CFL[t], 0) for t in range(1, m + 1)}
POF[50] = tf.reshape(POF[50],(100000,1))
POF[49] = tf.reshape(POF[49],(100000,1))


POF = tf.stack(list(POF.values()),axis=0)
#POF = tf.constant(POF, shape=(50,100000))

POF = tf.reshape(POF,(50,100000))
idx_payoffs = tf.math.argmax(POF > 0, axis=0)
FPOF = tf.transpose(tf.one_hot(idx_payoffs, m)) * POF
m_range = tf.constant(range(0, m), shape=(m,1),dtype=tf.float32)
dFPOF = FPOF * tf.exp(-r * m_range * Δt)

dFPOF = tf.reduce_sum(dFPOF)
PRICE = dFPOF / n
print(PRICE)




tf.Tensor(4.487445, shape=(), dtype=float32)
